In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download case3-datasaur-photo

100% 0.99G/0.99G [00:47<00:00, 22.8MB/s]
100% 0.99G/0.99G [00:48<00:00, 22.2MB/s]


In [ ]:
!unzip case3-datasaur-photo.zip

In [ ]:
import os
import shutil
import random

sources = ['/content/techosmotr/techosmotr/train/fictivniye(fictitious)/3-from-the-screen+photoshop',
           '/content/techosmotr/techosmotr/train/fictivniye(fictitious)/1-not-on-the-brake-stand',
           '/content/techosmotr/techosmotr/train/fictivniye(fictitious)/2-from-the-screen',
           '/content/techosmotr/techosmotr/train/fictivniye(fictitious)/4-photoshop'
          ]
destination = '/content/techosmotr/techosmotr/train/1-incorrect'


for source in sources:
  # gather all files
  allfiles = os.listdir(source)

  # iterate on all files to move them to destination folder
  for f in allfiles:

      #f = os.listdir(source)[random.randint(0, 300)]
      src_path = os.path.join(source, f)
      dst_path = os.path.join(destination, f)
      shutil.move(src_path, dst_path)

In [ ]:
source = '/content/techosmotr/techosmotr/train/1-incorrect'
destination = '/content/techosmotr/techosmotr/test/1-incorrect'

# gather all files
allfiles = os.listdir(source)


# iterate on all files to move them to destination folder
for i in range(0, 400):


    f = os.listdir(source)
    image = f[random.randint(0, len(f) - 1)]
    src_path = os.path.join(source, image)
    dst_path = os.path.join(destination, image)
    shutil.move(src_path, dst_path)

In [ ]:
len(os.listdir('/content/techosmotr/techosmotr/train/1-incorrect'))

In [ ]:
!rm -r "/content/techosmotr/techosmotr/test/.ipynb_checkpoints"

Resnet classification

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
import cv2
import json
import glob
from tqdm.notebook import tqdm

In [ ]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd, "/content/techosmotr/techosmotr/train")

labels = {}

for folder in os.listdir(ROOT_DIR):
    for file in os.listdir(os.path.join(ROOT_DIR, folder)):
        if file.endswith(".jpeg"):
            full_name = os.path.join(ROOT_DIR, folder, file)
            labels[full_name] = folder

files = labels.keys()

In [ ]:
files

In [ ]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vitl14_lc")
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
dinov2_vits14.to(device)
transform_image = T.Compose([T.ToTensor(), T.Resize(520), T.CenterCrop(518), T.Normalize([0.5], [0.5])])


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth
100%|██████████| 1.13G/1.13G [00:56<00:00, 21.4MB/s]
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_linear4_head.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_linear4_head.pth
100%|██████████| 19.5M/19.5M [00:01<00:00, 10.6MB/s]


In [ ]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
      for i, file in enumerate(tqdm(files)):
        embeddings = dinov2_vits14(load_image(file).to(device))

        all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

In [ ]:
embeddings = compute_embeddings(files)

  0%|          | 0/6585 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
y = [labels[file] for file in files]
embedding_list = list(embeddings.values())
X = np.array(embedding_list).reshape(-1, 1000)

In [ ]:
len(embedding_list)

6585

In [ ]:
X.shape

(6585, 1000)

Grid search

In [ ]:
parameters = param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

cv = GridSearchCV(BaggingClassifier(estimator=svm.SVC(gamma=0.001, kernel='poly', C=0.1), n_estimators=11), parameters, n_jobs=5)
cv.fit(X, y)

In [ ]:
print(f"best_score{cv.best_score_}")
print(f"best_score{cv.best_params_}")

best_score0.9466970387243736
best_score{'C': 0.1, 'gamma': 0.001, 'kernel': 'poly'}


halving_cv

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# param_grid = {
# "max_depth": [3, 4, 7, 10, 25],
# "gamma": [0.5, 1, 5, 10, 25],
# "min_child_weight": [1, 3, 5, 10, 25],
# "reg_lambda": [5, 10, 50, 100, 300],
# "scale_pos_weight": [1, 3, 5, 10, 25]
# }
# Grid Search CV implementation
xgb_cl = xgb.XGBClassifier(objective="binary:logistic")
xgb_cl.fit(X, le.fit_transform(y))


# print(f"best_score{halving_cv.best_score_}")
# print(f"best_score{halving_cv.best_params_}")

cross valid

dinov2_vitl14_lc + resize 518

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# xgb_cl = xgb.XGBClassifier(objective="binary:logistic")
# model = LogisticRegression(max_iter=500)

clf = BaggingClassifier(estimator=svm.SVC(gamma="scale", kernel='poly', C=0.1, probability=True), n_estimators=3)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.92027335, 0.93014427, 0.91571754, 0.91116173, 0.92558846])

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

xgb_cl = xgb.XGBClassifier(objective="binary:logistic")
scores = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores

RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1)

submission generation

In [ ]:
from PIL import Image
import pandas as pd

clf = BaggingClassifier(estimator=svm.SVC(gamma=0.001, kernel='poly', C=0.1), n_estimators=5)
clf.fit(X, y)


source = '/content/test'
allfiles = os.listdir(source)
test_data = {"file_index": [],
             "class": []}
  # iterate on all files to move them to destination folder
for f in allfiles:

  #f = os.listdir(source)[random.randint(0, 300)]
  src_path = os.path.join(source, f)

  input = load_image(os.path.join(source, f))
  embedding = dinov2_vits14(input.to(device))

  prediction = clf.predict(embedding[0].cpu().detach().numpy().reshape(1, -1))
  if prediction == '0-correct':
    prediction = 0
  else:
    prediction = 1

  test_data["file_index"].append(f.split(".")[0])
  test_data["class"].append(prediction)

In [ ]:
sub = pd.DataFrame(test_data)

In [ ]:
sub.to_csv("bag_lc_l_0964.csv", index=False)

In [ ]:
import pickle

filename = "bag_lc_l_0964.pickle"

# save model
pickle.dump(clf, open(filename, "wb"))